In [1]:
from openpyxl import load_workbook
import pandas as pd
from kopyt import Parser, node

In [2]:
!pip install -U kopyt

In [3]:
df = pd.read_excel('FinalDataset-test0.xlsx', sheet_name = 'data' )

In [4]:
print(df.columns)

Index(['package', 'xml', 'png', 'resourceId', 'filePath', 'location'], dtype='object')


In [5]:
def getfileContent(path):
    f = open(path, "r")
    content = f.read()
    
    return content

In [6]:
def nested(result, line):
    
    
    finalCode = ""
    for declaration in result: 
        # print(str(declaration.name))
        if( isinstance(declaration, node.FunctionDeclaration) and line > declaration.position.line):
            finalCode = str(declaration).replace( str(declaration.body), '')
        
        elif( isinstance(declaration, node.PropertyDeclaration) and line == declaration.position.line):
            return str(declaration)
        
        elif( isinstance(declaration, node.EnumDeclaration) and line > declaration.position.line):
            ffinalCode = str(declaration).replace( str(declaration.body), '')
        
        elif( isinstance(declaration, node.AnonymousInitializer) and line > declaration.position.line):
            finalCode = str(declaration).replace( str(declaration.body), '')
        
        elif( isinstance(declaration, node.ClassDeclaration) and line >declaration.position.line and declaration.body is not None):
            finalCode = nested( declaration.body.members , line)
            
        elif( isinstance(declaration, node.CompanionObject) and line > declaration.position.line and declaration.body is not None):
            finalCode = nested( declaration.body.members , line)
    
    return finalCode


def getResultForKotlin(code, line):
    
    
    parser = Parser(code)
    result = parser.parse()
    
    finalCode = ""

    # print(type(result), type(result.declarations))
    for declaration in result.declarations: 
        
        # print(type(declaration))
        if( isinstance(declaration, node.FunctionDeclaration) and line > declaration.position.line):
            finalCode = str(declaration).replace( str(declaration.body), '')
        
        elif( isinstance(declaration, node.PropertyDeclaration) and line == declaration.position.line):
            return str(declaration)
        
        elif( isinstance(declaration, node.AnonymousInitializer) and line > declaration.position.line):
            finalCode = str(declaration).replace( str(declaration.body), '')
            
        elif( isinstance(declaration, node.EnumDeclaration) and line > declaration.position.line):
            finalCode = str(declaration).replace( str(declaration.body), '')
            
        elif( isinstance(declaration, node.ClassDeclaration) and line > declaration.position.line and declaration.body is not None):
            finalCode = nested( declaration.body.members , line)
            
        elif( isinstance(declaration, node.CompanionObject) and line > declaration.position.line and declaration.body is not None):
            # finalCode = str(declaration)
            finalCode = nested( declaration.body.members , line)
       
    
    #Excludes tests
    if finalCode.startswith('@ParameterizedTest') or finalCode.startswith('@Test'):
        return ''
    else:
        return finalCode.split('=')[0].strip()

In [7]:
def formatResourceId(resourceId: str):
    words = resourceId.split('_')
    result: str = ""
    for i in enumerate(words):
        if( i[0] == 0 ):
            result = result + i[1]
        else:
            result = result + i[1].capitalize()
    
    return result

In [8]:
def filterOverlappingName( data, column, resourceId):
    column = column - 1
    
    # print(len(data), column)
    if( column == 0 ):
        return True
    if data[column-1].isalpha() == True:
        return False
    if data[column-1] == '_':
        return False
    
    
    if len(data) <= column + len(resourceId):
        return True;
    if data[column + len(resourceId)].isalpha() == True:
        return False
    if data[column + len(resourceId)] == '_':
        return False
    
    return True
    

In [9]:
print(filterOverlappingName("previous", 1, "previous"))
print(filterOverlappingName("  previous", 3, "previous"))
print(filterOverlappingName("  MyPrevious", 5, "previous"))
print(filterOverlappingName("  M.Previous", 5, "previous"))
print(filterOverlappingName(".previous  ", 2, "previous"))
print(filterOverlappingName("previous  ", 1, "previous"))
print(filterOverlappingName(".previousDay", 2, "previous"))
print(filterOverlappingName(".previous ", 2, "previous"))
print(filterOverlappingName(".previous.m", 2, "previous"))
print(filterOverlappingName("l.previous", 3, "previous"))

True
True
False
True
True
True
False
True
True
True


In [ ]:
result = []
counter = 0
failed = 0

for index, row in df.iterrows():
    
    path = row['filePath']
    line = int( row['location'].split(',')[0].split('=')[-1])
    col = int(row['location'].split(',')[1].split('=')[-1])
    resourceId = formatResourceId(row['resourceId'].split('/')[-1])
    
    
    code = getfileContent(path)
    
        
    if path.split('.')[-1] == 'kt':
        
        try:
            if filterOverlappingName(code.split('\n')[ int(line)-1 ], col, resourceId) == False:
                continue
        except IndexError: 
            failed += 1
            print('Index Error', resourceId, path, col, failed)
            
        counter += 1
        if(counter % 500 == 0):
            print("Index:", index, "parsed:", counter, "Failed:", failed)
        
        cols = row.tolist()
        cols.append(getResultForKotlin(code, int(line)))
        result.append( cols)

        


In [11]:

kt = pd.DataFrame(result)

In [14]:
kt.to_excel( 'kotlin-mapping-short.xlsx', sheet_name='data', index = False)

In [15]:
len(kt)

257543

In [19]:
len(kt[6].unique())

738

In [24]:
len(kt[3].unique())

100

In [55]:
kt.columns = ['package', 'xml-path', 'png-path', 'resource-id', 'source-path','location', 'code']
kt['package'].unique()

array(['com.byagowi.persiancalendar', 'com.iven.musicplayergo',
       'com.greenaddress.greenbits_android_wallet', 'ws.xsoh.etar',
       'com.duckduckgo.mobile.android', 'com.better.alarm', 'com.fsck.k9'],
      dtype=object)

In [77]:
code_to_code_df = pd.DataFrame( kt[['package', 'resource-id', 'code']] )

In [78]:
code_to_code_df

,package,resource-id,code
0,com.byagowi.persiancalendar,com.byagowi.persiancalendar:id/previous,init
1,com.byagowi.persiancalendar,com.byagowi.persiancalendar:id/next,init
2,com.byagowi.persiancalendar,com.byagowi.persiancalendar:id/next,
3,com.byagowi.persiancalendar,com.byagowi.persiancalendar:id/next,init
4,com.byagowi.persiancalendar,com.byagowi.persiancalendar:id/next,init
...,...,...,...
257538,com.fsck.k9,com.fsck.k9:id/next,"@Throws(XmlPullParserException::class, IOExcep..."
257539,com.fsck.k9,com.fsck.k9:id/next,private fun configureMenu(menu: Menu?)
257540,com.fsck.k9,com.fsck.k9:id/next,private fun configureMenu(menu: Menu?)
257541,com.fsck.k9,com.fsck.k9:id/next,private fun configureMenu(menu: Menu?)


In [79]:
for index, row in code_to_code_df.iterrows():
    
    if not row['code']:
        row['code'] = None
    
    if row['resource-id']:
        row['resource-id'] = row['resource-id'].split('/')[-1]
    
    row['package'] = row['package'].split('.')[-1]

In [80]:
code_to_code_df = code_to_code_df.dropna()

In [81]:
code_to_code_df['package'].unique()

array(['persiancalendar', 'musicplayergo', 'greenbits_android_wallet',
       'etar', 'android', 'alarm', 'k9'], dtype=object)

In [82]:
category = ['Time', 'Multimedia', 'Money', 'Time', 'Internet', 'System', 'Science&Education']
names: list = list(code_to_code_df['package'].unique())

In [83]:
#Replace all rpackage with respective category

for index, row in code_to_code_df.iterrows():
    row['package'] =  category[names.index(row['package'])]

In [84]:
code_to_code_df.columns = ['category', 'resource-id', 'code']

In [85]:
len(code_to_code_df['category'].unique())

6

In [86]:
code_to_code_df.to_excel( 'kotlin-code-code.xlsx', sheet_name='data', index = False)